In [1]:
import boto3
import pandas as pd
import json
import time
from sklearn.metrics import f1_score, balanced_accuracy_score

In [2]:
# AWS credentials
with open("../resources/credentials.json", "r") as f:
    creds = json.load(f)
    
ACCESS_KEY = creds["Access key ID"]
SECRET_KEY = creds["Secret access key"]
region = "eu-west-1"

# Clients
lex_runtime = "lex-runtime"
lex_models = "lex-models"

# Bot details
with open("../resources/bot_config.json", "r") as f:
    bot_config = json.load(f)
    
bot_name = bot_config["bot_name"]
bot_alias = bot_config["bot_alias"]

In [3]:
client = boto3.client(lex_runtime,
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY,
                      region_name=region)

In [4]:
# Predict function
def lex_predict(df_, client, bot_name, bot_alias):
    
    df_["lex_pred"] = ""
    df_["lex_prob"] = ""

    for i, row in df_.iterrows():
        print(f"\rProcessing {i+1} of {len(df_)}", end="")
        
        text = row.text
        response = client.post_text(
            botName=bot_name,
            botAlias=bot_alias,
            userId='test_user',
            sessionAttributes={
                'sessionID': 'test_session'
            },
            inputText=text,
        )
        
        try:
            df_.loc[:,"lex_pred"].iloc[i] = response["intentName"]
            df_.loc[:, "lex_prob"].iloc[i] = response["nluIntentConfidence"]['score']
            
        except KeyError:
            df_.loc[:,"lex_pred"].iloc[i] = "None"
            continue   
    
    return df_

# Test on standard test test

In [5]:
# Read training intents/utterances
df_small = pd.read_excel("../data/test.xlsx")
df_small.head(2)

,text,label,intent
0,How do I locate my card,11,card_arrival
1,I still have not received my new card I ordere...,11,card_arrival


In [7]:
df_test1 = lex_predict(df_small.copy(), client, bot_name, bot_alias)

Processing 1 of 3079

C:\Users\Giebels\anaconda3\envs\py_boto\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Processing 110 of 3079

In [ ]:
df_test1["correct"] = (df_test1["intent"] == df_test1["lex_pred"]).astype(int)
thresh = 0.95

# Lex accuracy
acc = df_test1['correct'].mean()
f1_ = f1_score(df_test1["intent"], df_test1["lex_pred"], average='weighted')
conf_acc = df_test1.loc[df_test1['lex_prob'].replace("","0").astype(float)>=thresh, 'correct'].mean()
f1_conf = f1_score(df_test1.loc[df_test1['lex_prob'].replace("","0").astype(float)>=thresh, "intent"], 
                   df_test1.loc[df_test1['lex_prob'].replace("","0").astype(float)>=thresh, "lex_pred"], average='weighted')
n_conf = len(df_test1.loc[df_test1['lex_prob'].replace("","0").astype(float)>=thresh, :])

print(f"Overall Accuracy: {acc}")
print(f"F1 Score: {f1_}")
print(f"95% + confidence Accuracy ({n_conf} samples): {conf_acc}")
print(f"95% + confidence F1 Score ({n_conf} samples): {f1_conf}")

Overall Accuracy: 0.8713868139006171
F1 Score: 0.8743884033215368
95% + confidence Accuracy (2152 samples): 0.9744423791821561
95% + confidence F1 Score (2152 samples): 0.9742904026379858


In [ ]:

df_test1.to_excel("../data/lex_results.xlsx", index=False)


# Test on large test test

In [ ]:
# Read training intents/utterances
df_large = pd.read_excel("../data/test_massive.xlsx")
df_large.head(2)

,text,label,intent
0,When did you send me my new card,11,card_arrival
1,Do you have info about the card on delivery,11,card_arrival


In [ ]:
df_test2 = lex_predict(df_large.copy(), client, bot_name, bot_alias)

Processing 1 of 10178

C:\Users\Giebels\anaconda3\envs\py_boto\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Processing 10178 of 10178

In [ ]:
df_test2["correct"] = (df_test2["intent"] == df_test2["lex_pred"]).astype(int)
thresh = 0.95

# Lex accuracy
acc = df_test2['correct'].mean()
f1_ = f1_score(df_test2["intent"], df_test2["lex_pred"], average='weighted')
conf_acc = df_test2.loc[df_test2['lex_prob'].replace("","0").astype(float)>=thresh, 'correct'].mean()
f1_conf = f1_score(df_test2.loc[df_test2['lex_prob'].replace("","0").astype(float)>=thresh, "intent"], 
                   df_test2.loc[df_test2['lex_prob'].replace("","0").astype(float)>=thresh, "lex_pred"], average='weighted')
n_conf = len(df_test2.loc[df_test2['lex_prob'].replace("","0").astype(float)>=thresh, :])

print(f"Overall Accuracy: {acc}")
print(f"F1 Score: {f1_}")
print(f"95% + confidence Accuracy ({n_conf} samples): {conf_acc}")
print(f"95% + confidence F1 Score ({n_conf} samples): {f1_conf}")

Overall Accuracy: 0.8639221851051287
F1 Score: 0.8663328163664185
95% + confidence Accuracy (6939 samples): 0.975789018590575
95% + confidence F1 Score (6939 samples): 0.9757309753879153


In [ ]:
df_test2.to_excel("../data/lex_results_massive.xlsx", index=False)